In [1]:
import pybnb
import numpy as np
from utils import *
import operator
from collections import defaultdict

ms_package_path = '/home/frashidi/software/bin/ms'
csp_solver_path = '/home/frashidi/software/temp/csp_solvers/maxino/code/build/release/maxino'

In [2]:
ground, noisy, (countFN,countFP,countNA) = get_data(n=10, m=8, seed=1, fn=0.20, fp=0, na=0, 
                                                    ms_package_path=ms_package_path)
# print(noisy)

solution, (flips_0_1, flips_1_0, flips_2_0, flips_2_1) = PhISCS_I(noisy, beta=0.20, alpha=0.0001)
# print(solution)

# solution = PhISCS_B(noisy, beta=0.20, alpha=0.0000000001, csp_solver_path=csp_solver_path)
# print(solution)

print(np.where(solution != noisy))

(array([3]), array([2]))


In [3]:
def is_conflict_free_gusfield(I):
    def sort_bin(a):
        b = np.transpose(a)
        b_view = np.ascontiguousarray(b).view(np.dtype((np.void, b.dtype.itemsize * b.shape[1])))
        c = b[np.argsort(b_view.ravel())[::-1]]
        return np.transpose(c)

    O = sort_bin(I)
    #delete duplicate columns
    #print(O, '\n')
    Lij = np.zeros(O.shape, dtype=int)
    for i in range(O.shape[0]):
        maxK = 0
        for j in range(O.shape[1]):
            if O[i,j] == 1:
                Lij[i,j] = maxK
                maxK = j+1
    #print(Lij, '\n')
    Lj = np.amax(Lij, axis=0)
    #print(Lj, '\n')

    for i in range(O.shape[0]):
        for j in range(O.shape[1]):
            if O[i,j] == 1:
                if Lij[i,j] != Lj[j]:
                    return False
    return True

In [4]:
def get_lower_bound(noisy, partition_randomly=False):
    def important_pair_of_columns_in_conflict(D):
        important_columns = defaultdict(lambda: 0)
        for p in range(D.shape[1]):
            for q in range(p + 1, D.shape[1]):
                oneone = 0
                zeroone = 0
                onezero = 0
                for r in range(D.shape[0]):
                    if D[r,p] == 1 and D[r,q] == 1:
                        oneone += 1
                    if D[r,p] == 0 and D[r,q] == 1:
                        zeroone += 1
                    if D[r,p] == 1 and D[r,q] == 0:
                        onezero += 1
                if oneone*zeroone*onezero > 0:
                    important_columns[(p,q)] += oneone*zeroone*onezero
        return important_columns
    
    def get_partition_sophisticated(D):
        ipofic = important_pair_of_columns_in_conflict(D)
        if len(ipofic) == 0:
            return []
        sorted_ipofic = sorted(ipofic.items(), key=operator.itemgetter(1), reverse=True)
        pairs = [sorted_ipofic[0][0]]
        elements = [sorted_ipofic[0][0][0], sorted_ipofic[0][0][1]]
        sorted_ipofic.remove(sorted_ipofic[0])
        for x in sorted_ipofic[:]:
            notFound = True
            for y in x[0]:
                if y in elements:
                    sorted_ipofic.remove(x)
                    notFound = False
                    break
            if notFound:
                pairs.append(x[0])
                elements.append(x[0][0])
                elements.append(x[0][1])
        #print(sorted_ipofic, pairs, elements)
        partitions = []
        for x in pairs:
            partitions.append(D[:,x])
        return partitions
    
    def get_partition_random(D):
        d = int(D.shape[1]/2)
        partitions_id = np.random.choice(range(D.shape[1]), size=(d, 2), replace=False)
        partitions = []
        for x in partitions_id:
            partitions.append(D[:,x])
        return partitions
    
    def lower_bound_for_a_pair_of_columns(D):
        foundOneOne = False
        numberOfZeroOne = 0
        numberOfOneZero = 1
        for r in range(D.shape[0]):
            if D[r,0] == 1 and D[r,1] == 1:
                foundOneOne = True
            if D[r,0] == 0 and D[r,1] == 1:
                numberOfZeroOne += 1
            if D[r,0] == 1 and D[r,1] == 0:
                numberOfOneZero += 1
        if foundOneOne:
            if numberOfZeroOne*numberOfOneZero > 0:
                return min(numberOfZeroOne, numberOfOneZero)
        return 0
    
    LB = []
    if partition_randomly:
        partitions = get_partition_random(noisy)
    else:
        partitions = get_partition_sophisticated(noisy)
    for D in partitions:
        LB.append(lower_bound_for_a_pair_of_columns(D))
    return sum(LB)

In [5]:
noisy = np.array([
    [0,1,1,0],
    [1,0,0,1],
    [1,1,0,0],
    [0,0,1,0]
])

In [6]:
class PhISCS(pybnb.Problem):
    def __init__(self, I):
        self.I = I
        self.X = np.where(self.I == 0)
        self.flip = 0
        self.idx = 0
    
    def sense(self):
        return pybnb.minimize

    def objective(self):
        if is_conflict_free_gusfield(self.I):
            return self.flip
        else:
            return pybnb.Problem.infeasible_objective(self)

    def bound(self):
        return self.flip + get_lower_bound(self.I, partition_randomly=True)

    def save_state(self, node):
        node.state = (self.I, self.idx, self.flip)

    def load_state(self, node):
        self.I, self.idx, self.flip = node.state

    def branch(self):
        if self.idx < len(self.X[0]):
            node = pybnb.Node()
            I = self.I.copy()
            x = self.X[0][self.idx]
            y = self.X[1][self.idx]
            I[x, y] = 1
            node.state = (I, self.idx+1, self.flip+1)
            yield node
            
            node = pybnb.Node()
            I = self.I.copy()
            x = self.X[0][self.idx]
            y = self.X[1][self.idx]
            I[x, y] = 0
            node.state = (I, self.idx+1, self.flip)
            yield node

problem = PhISCS(noisy)
results = pybnb.solve(problem)

print(results.best_node.state)

Starting branch & bound solve:
 - dispatcher pid: 120734 (phi.cs.indiana.edu)
 - worker processes: 1
--------------------------------------------------------------------------------------------------------------------------
         Nodes        |                      Objective Bounds                       |              Work              
      Expl    Unexpl  |      Incumbent           Bound    Rel. Gap         Abs. Gap | Time (s)  Nodes/Sec Imbalance   Idle
         0         1  |            inf            -inf         inf%             inf |      0.0       0.00     0.00%      0
         1         2  |            inf               1         inf%             inf |      0.0     277.31     0.00%      0
*       74         1  |              2               0  100.000000%               2 |      0.1    1282.20     0.00%      0
        75         0  |              2               0  100.000000%               2 |      0.1     528.17     0.00%      0
-------------------------------------------

In [11]:
a = np.array([
    [0, 0, 1, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 1, 0, 1],
    [0, 1, 1, 1, 0, 0, 1, 0],
    [0, 0, 1, 1, 0, 0, 1, 0],
    [0, 0, 1, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 1, 0],
    [1, 0, 1, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]    
])
is_conflict_free_farid(a)
is_conflict_free_gusfield(a)

True

–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

In [36]:
def important_columns(D):
    ic = defaultdict(lambda: 0)
    for p in range(D.shape[1]):
        for q in range(p + 1, D.shape[1]):
            conf_oneone = []
            conf_zeroone = []
            conf_onezero = []
            for r in range(D.shape[0]):
                if D[r][p] == 1 and D[r][q] == 1:
                    conf_oneone.append(r)
                if D[r][p] == 0 and D[r][q] == 1:
                    conf_zeroone.append(r)
                if D[r][p] == 1 and D[r][q] == 0:
                    conf_onezero.append(r)
            for r1 in conf_oneone:
                for r2 in conf_zeroone:
                    for r3 in conf_onezero:
                        print(p,q, r1, r2, r3)
                        ic[p] += 1
                        ic[q] += 1
    return sorted(ic.items(), key=operator.itemgetter(1), reverse=True)

print(important_columns(noisy))

1 2 0 3 1
1 2 0 3 2
[(1, 2), (2, 2)]


In [16]:
# import pandas as pd

# df = pd.DataFrame(noisy)
# df.columns = ['mut'+str(i) for i in range(noisy.shape[1])]
# df.index = ['cell'+str(i) for i in range(noisy.shape[0])]
# df.index.name = 'cellID/mutID'
# df.to_csv('noisy.SC', sep='\t')

# df = pd.read_csv(file, index_col=0, sep='\t')

[1] [ms package (paper)](https://academic.oup.com/bioinformatics/article/18/2/337/225783)  
[2] [ms package (download)](http://home.uchicago.edu/~rhudson1/source/mksamples.html)  
[3] [csp solvers](http://mse17.cs.helsinki.fi/descriptions.html)  
[4] [PhISCS](https://www.biorxiv.org/content/early/2018/07/25/376996)  

For installing maxino go to the url of [3] and then `wget` the `maxino` package. Then `unzip` it. Ater that go to the code folder and just run `make`. Then change `csp_solver_path` accordingly.  

n = number of cells  
m = number of mutations  
seed = is a seed number of generating the ground truth by ms package (not important leave it as 1)  
fn = false negative rate  
fp = false positive rate  
na = missing value rate